This notebook is for running a first try of BOWave on the Frolich et. al data to see if we match their paper's results.
This requires 16 < x < 32 gb of RAM. Recommend running on Caviness with --mem-per-cpu=32gb flag set.

#Load ICs

In [12]:
%pip install matplotlib
%pip install scikit-learn

  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/c2/da/a5622266952ab05dc3995d77689cba600e49ea9d6c51d469c077695cb719/matplotlib-3.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/aa/55/02c6d24804592b862b38a85c9b3283edc245081390a520ccd11697b6b24f/contourpy-1.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/2b/e8/61b8525acf26ec222518bdff127ae502bfa3408981fb5e5493f2b037d7fb/fonttools-4.42.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.0/151.0 kB 11.1 MB/s eta 0:00:00
  Obtaining dependency information for kiwisolver>=1.0.1 from https://files.pythonhosted.org/packages/6f/40/4ab

In [5]:
import pyrootutils

pyrootutils.set_root(path='/work/cniel/ajmeek/BOWaves/BOWaves', pythonpath=True)
#path = pyrootutils.find_root(search_from=__file__, indicator=".git")
#pyrootutils.set_root(path=path, pythonpath=True)

import BOWaves.utilities.dataloaders as dataloaders
import os
import numpy as np

In [6]:
#frolich_ics = {'ICs': np.array([]), 'labels': np.array([])}
frolich_ics = {'ICs': [], 'labels': []}

#for file in directory frolich data
frolich_data = os.listdir('../data/frolich')

#filter out subdirectories such as /img
frolich_data = [file for file in frolich_data if not os.path.isdir(file)]

for file in frolich_data:
    ICs, labels = dataloaders.load_and_visualize_mat_file_frolich('../data/frolich/' + file, visualize=False)
    frolich_ics['ICs'].extend(ICs)
    frolich_ics['labels'].extend(labels)


Now create codebooks since we have the ICs and their labels.

First split off 20% for testing.

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(frolich_ics['ICs'], frolich_ics['labels'], test_size=0.2, random_state=42)

Now out of the training set, split into the different classes. Frolich's data has 4 classes.

In [10]:
if len(X_train) != len(y_train):
    raise ValueError('X_train and y_train are not the same length.')

all_classes = ['neural', 'blink', 'muscle', 'mixed', 'lateyes', 'heart']

# Forgot what the classes were. check on Caviness
neural = {'ICs': [], 'centroids': [], 'labels': [], 'shifts': [], 'distances': [], 'inertia': []}
blink = {'ICs': [], 'centroids': [], 'labels': [], 'shifts': [], 'distances': [], 'inertia': []}
muscle = {'ICs': [], 'centroids': [], 'labels': [], 'shifts': [], 'distances': [], 'inertia': []}
mixed = {'ICs': [], 'centroids': [], 'labels': [], 'shifts': [], 'distances': [], 'inertia': []}
lateyes = {'ICs': [], 'centroids': [], 'labels': [], 'shifts': [], 'distances': [], 'inertia': []}
heart = {'ICs': [], 'centroids': [], 'labels': [], 'shifts': [], 'distances': [], 'inertia': []}

for i in range(len(X_train)):
    if y_train[i] == 'neural':
        neural['ICs'].append(X_train[i])
    elif y_train[i] == 'blink':
        blink['ICs'].append(X_train[i])
    elif y_train[i] == 'muscle':
        muscle['ICs'].append(X_train[i])
    elif y_train[i] == 'mixed':
        mixed['ICs'].append(X_train[i])
    elif y_train[i] == 'lateyes':
        lateyes['ICs'].append(X_train[i])
    elif y_train[i] == 'heart':
        heart['ICs'].append(X_train[i])
    else:
        raise ValueError('Unknown class label: ' + y_train[i])

So Dr. B raised a really good point that I had forgotten. Sikmeans does not take just raw ICs, it takes random windows sampled from throughout the ICs. I need to better remember the details of the paper. Will reread it now, but also I need to rework the below and take windows from them first before passing to sikmeans.

So the first argument to sikmeans should be of shape (# ICs, window length).

His function where he takes window slices out of them is in the dataloaders file. A bit of odd organization and I looked right over it. Just have that in a new window below.

He even says "ICs from diff subjects have diff lengths so don't straightforwardly concatenate into an array" and I missed that. My error.

Taking windows below:

In [ ]:
#grab windows from the ICs
#not all ICs have the same length, so must take windows. This is what we pass in to sikmeans anyways. Different windows for each class.

#what is the window len? Set hyperparams above. Later on will modify to be chosen from grid search in nested cross validation.
window_len = 256 #what's the sampling rate again?

windows_per_class = {'neural': [], 'blink': [], 'muscle': [], 'mixed': [], 'lateyes': [], 'heart': []}

tot_num_windows = 0 #to house total number of windows for all ICs
for label in all_classes:
    #iterate through class and collect info about sequence lengths
    ic_lengths = []
    n_ics = len(label['ICs'])
    for ic in label['ICs']:
        ic_lengths.append(len(ic))

    #Currently, assuming that we are not taking a subset of the ICs at all. Carlos had the option for that in his earlier window code.
    #So the number of windows per ic will just be the length of each ic / win len.
    n_windows_per_ic = [ic_len // window_len for ic_len in ic_lengths]
    tot_num_windows += sum(n_windows_per_ic)

    #Now that we have the number of windows per ic, we can create the windows.

    rng = np.random.RandomState(42)

    X = np.zeros((tot_num_windows, window_len)) #X is for each class. Stack later
    win_start = 0
    for label in all_classes:
        for ic in label['ICs']:
            windows_per_ic = len(ic) // window_len
            time_idx = np.arange(0, len(ic)-window_len+1, window_len)
            time_idx = rng.choice(time_idx, size=windows_per_ic, replace=False)
            time_idx = time_idx[:, None] + np.arange(window_len)[None, :]
            X[win_start:win_start+windows_per_ic] = ic[time_idx]
            win_start += windows_per_ic

    windows_per_class[label] = X

X, calculated above, should now contain all windows from all ICs. Now we can pass this to sikmeans.
Actually hold on. It needs to be done by class. Done. So now pass windows_per_class[label] to sikmeans in as the first parameter.

Note - this may get to the point where I can't hold these in memory any longer.

In [17]:
from BOWaves.sikmeans.sikmeans_core import shift_invariant_k_means
metric, init = 'cosine', 'random'
num_clusters = 16
centroid_len = 256
n_runs = 3
n_jobs = 1
rng = 42#np.random.RandomState(42)

#need to do this per class.
neural['centroids'], neural['labels'], neural['shifts'], neural['distances'], neural['inertia'], _ = shift_invariant_k_means(windows_per_class['neural'], num_clusters, centroid_len, metric=metric, init=init, n_init=n_runs, rng=rng,  verbose=True)

blink['centroids'], blink['labels'], blink['shifts'], blink['distances'], blink['inertia'], _ = shift_invariant_k_means(windows_per_class['blink'], num_clusters, centroid_len, metric=metric, init=init, n_init=n_runs, rng=rng,  verbose=True)

muscle['centroids'], muscle['labels'], muscle['shifts'], muscle['distances'], muscle['inertia'], _ = shift_invariant_k_means(windows_per_class['muscle'], num_clusters, centroid_len, metric=metric, init=init, n_init=n_runs, rng=rng,  verbose=True)

mixed['centroids'], mixed['labels'], mixed['shifts'], mixed['distances'], mixed['inertia'], _ = shift_invariant_k_means(windows_per_class['mixed'], num_clusters, centroid_len, metric=metric, init=init, n_init=n_runs, rng=rng,  verbose=True)

lateyes['centroids'], lateyes['labels'], lateyes['shifts'], lateyes['distances'], lateyes['inertia'], _ = shift_invariant_k_means(windows_per_class['lateyes'], num_clusters, centroid_len, metric=metric, init=init, n_init=n_runs, rng=rng,  verbose=True)

heart['centroids'], heart['labels'], heart['shifts'], heart['distances'], heart['inertia'], _ = shift_invariant_k_means(windows_per_class['heart'], num_clusters, centroid_len, metric=metric, init=init, n_init=n_runs, rng=rng,  verbose=True)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (210,) + inhomogeneous part.

Now that we have the codebooks, let's run the bowav clf code.
We want to do leave one subject out cross validation to try and classify the labels on the held out test set.

First, define BOWav to create the bag-of-words representations of the features learned in the codebooks.

Sanity checks before continuing.

windows_per_class, indexed by the labels, holds my codebooks.

Each element should be of the shape (k, P) with k and P being the number of centroids and centroid length, respectively.

Taken from Carlos' comment:
        A sequence of codebooks. C[i].shape = (k, P), with k and P being the number of centroids and centroid lenght, respectively.

The length of the centroids should be exactly equal to the window length, since each centroid is just a representative waveform. I.e., sikmeans just picks one of the data points to be most central, not that it's combining data points to get the mean of the prob distr or something.

Mistake - the windows per class are not the centroids. Sikmeans returns a certain number of centroids. If they were the same for each that wouldn't make much sense. How many does it return? Whatever the num of clusters is - above I have set to 16.



In [ ]:
#sanity check
for label in windows_per_class:
    print(label + ': ' + str(windows_per_class[label].shape))

In [ ]:
from BOWaves.sikmeans.sikmeans_core import _assignment_step

#X_train above should be my raw ICs.

#does this work? forgot list concatenation syntax
codebooks = neural['centroids'] + blink['centroids'] + muscle['centroids'] + mixed['centroids'] + lateyes['centroids'] + heart['centroids']

def bag_of_waves(raw_ics, codebooks):
    """
    Creates a bag-of-words representation of the input data using the codebooks.

    The codebooks should be a concatenation of all the centroids learned above, one for each class.

    Parameters
    ----------
    codebooks - the array of centroids. I.e., neural['centroids']

    Returns
    -------
    X: matrix of shape (n_ics, n_features)
        The bag-of-words representation of the input data.
        (note to self, this should definitely be a sparse matrix right?)
    """

    #n_ics is the number of ICs from all classes in the codebooks list
    n_ics = sum(len(codebook['ICs']) for codebook in codebooks)

    #n_centroids is the number of centroids from all classes in the codebooks list
    n_centroids = sum(len(codebook['centroids']) for codebook in codebooks)

    x_squared_norms = None
    X = np.zeros((n_ics, n_centroids), dtype=codebooks[0]['centroids'].dtype)

    for ic in range(n_ics):
        for centroid in range(n_centroids):
            nu, _, _ = _assignment_step(codebooks[centroid]['ICs'][ic], codebooks[centroid]['centroids'], x_squared_norms, metric='cosine')

            nu, counts = np.unique(nu, return_counts=True)

            i_feature = nu + centroid * n_centroids
            X[ic, i_feature] = counts
    return X



Now that we have the bag-of-words representation, we can run the classifier.

The above should all work, calculating codebooks and getting their bowav representation. The only problem that I foresee there besides the usual bugs are memory issues.

So how will I do the below? Use Carlos' code as a base. I need to get the subject indices for LOO. Then do grid search within that. But how can I get the subject if we're combining all the ICs into their different classes?
I could put in raw ICs from each subject in the bag_of_waves function. That'll get a per subject bowav representation, built on codebooks learned from all subjects. That should probably work. But, check the ICLR paper again to make sure.

In [ ]:
from sklearn.model_selection import LeaveOneOut, grid_search_cv #not sure if this is right
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from numpy.random import default_rng

#list of classifier hyperparameters


#get data here - want raw_ICs, codebooks. Split between subjects.
#note - should I make a new function for it as Carlos did?


#pipeline set up, params just what Carlos used
pipe = Pipeline([('scaler', TfidfTransformer()), ('clf', LogisticRegression())])

clf_params = dict(
    clf__class_weight='balanced',
    clf__solver='saga',
    clf__penalty='elasticnet', #Carlos' default. could also be none, l1, or l2
    clf__random_state=np.random.RandomState(13),
    clf__multi_class='multinomial',
    clf__warm_start=True,
    clf__max_iter=1000, #Carlos' default.
)
pipe.set_params(**clf_params)

candidate_params = dict(
    clf__C=[0.1,1,10], #regularization factor
    clf__l1_ratio=[0, 0.2, 0.4, 0.6, 0.8, 1],
)

results = grid_search_cv(
    pipe,
    candidate_params,
    X,
    y,
    cv,
    n_jobs=1 #parallelization
)